In [ ]:
# %pip install onnx_asr[gpu,hub]

In [2]:
import onnx_asr.preprocessors

providers = ["CPUExecutionProvider"]

preprocessors = [
    onnx_asr.preprocessors.Preprocessor(name, {"providers": providers})
    for name in ["gigaam_v2", "gigaam_v3", "kaldi", "kaldi_fast", "nemo80", "nemo128", "whisper80", "whisper128"]
]

resamplers = [onnx_asr.preprocessors.Resampler(rate, {"providers": providers}) for rate in [8_000, 16_000]]

In [1]:
import onnx_asr
from onnx_asr.utils import read_wav_files

model = onnx_asr.load_model("gigaam-v3-ctc", providers=["TensorrtExecutionProvider"], quantization="int8")

2025-12-19 11:44:33.739729330 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card0/device/vendor"
/home/vscode/.local/lib/python3.12/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:123: UserWarning: Specified provider 'TensorrtExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [ ]:
for name in ["kaldi", "kaldi_fast"]:
    preprocessor = onnx_asr.preprocessors.Preprocessor(name, {"providers": providers})
    resampler = onnx_asr.preprocessors.Resampler(16_000, {"providers": providers})
    for k in [1, 4]:
        print(f"Preprocessor: {name}, Batch size: {k}")

        data = ["../test30s.wav"] * k
        waveform = read_wav_files(data, 16_000)
        resampled = resampler(*waveform)
        preprocessed = preprocessor(*resampled)

        t1 = %timeit -o -q -n 10 resampler(*waveform)
        t2 = %timeit -o -q -n 10 preprocessor(*resampled)

        for res in [t1, t2]:
            res.timings = [t / k for t in res.timings]

        print(f"resampler: {t1}")
        print(f"preprocessor: {t2}")
        print()

In [ ]:
for k in [1, 4, 16]:
    print(f"Batch size: {k}")

    data = ["test.wav"] * k
    waveform = read_wav_files(data, 16_000)
    resampled = model.resampler(*waveform)
    preprocessed = model.asr._preprocessor(*resampled)
    encoded = model.asr._encode(*preprocessed)

    t1 = %timeit -o -q model.resampler(*waveform)
    t2 = %timeit -o -q model.asr._preprocessor(*resampled)
    t3 = %timeit -o -q model.asr._encode(*preprocessed)
    t4 = %timeit -o -q list(model.asr._decoding(*encoded, None))
    t5 = %timeit -o -q model.recognize(data)

    for res in [t1, t2, t3, t4, t5]:
        res.timings = [t / k for t in res.timings]

    print(f"resampler: {t1}")
    print(f"preprocessor: {t2}")
    print(f"encoder: {t3}")
    print(f"decoding: {t4}")
    print(f"full recognize: {t5}")
    print()